<a href="https://colab.research.google.com/github/brunorosilva/ai_especialization_usp/blob/master/IAD-004-Aprendizagem_de_Maquina_1/NaiveBayes_SpamFilter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix

# Construindo um Filtro de SPAM com um classificador *Naive Bayes*

O problema de filtragem de e-mails não-solicitados (SPAM) é um clássico do processamento de texto.

## Base de dados

A base [Enron-Spam](http://www2.aueb.gr/users/ion/data/enron-spam/) contém um conjunto de e-mails em inglês pré-rotulados como mensagens legítimas ("Ham") e indesejáveis ("Spam").

A célula abaixo recupera a base e descompacta-a.

## Base de dados

A base [Enron-Spam](http://www2.aueb.gr/users/ion/data/enron-spam/) contém um conjunto de e-mails em inglês pré-rotulados como mensagens legítimas ("Ham") e indesejáveis ("Spam").

A célula abaixo recupera a base e descompacta-a.

In [ ]:
!wget -O lingspam_public.tar.gz http://www.aueb.gr/users/ion/data/lingspam_public.tar.gz
!tar xzf lingspam_public.tar.gz

--2020-07-21 22:41:47--  http://www.aueb.gr/users/ion/data/lingspam_public.tar.gz
Resolving www.aueb.gr (www.aueb.gr)... 195.251.255.156
Connecting to www.aueb.gr (www.aueb.gr)|195.251.255.156|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://www2.aueb.gr/users/ion/data/lingspam_public.tar.gz [following]
--2020-07-21 22:41:48--  http://www2.aueb.gr/users/ion/data/lingspam_public.tar.gz
Resolving www2.aueb.gr (www2.aueb.gr)... 195.251.255.138
Connecting to www2.aueb.gr (www2.aueb.gr)|195.251.255.138|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11564714 (11M) [application/x-gzip]
Saving to: ‘lingspam_public.tar.gz’

lingspam_public.tar 100%[===================>]  11.03M  1.96MB/s    in 8.2s    

2020-07-21 22:41:57 (1.35 MB/s) - ‘lingspam_public.tar.gz’ saved [11564714/11564714]



A base tem os subdiretórios ```bare```, ```lemm```, ```lemm_stop``` e ```stop```.
Vamos usar neste exercício o subdiretório ```lemm_stop``` que pré-processa as mensagens de forma a remover palavras excessivamente comuns e sub-variedades  gramaticais (e.g.: plural).

As mensagens estão sub-divididas de forma aleatória em 10 sub-diretórios.

Mensagens indesejadas têm o seu nome iniciado por ```spm```.

O código abaixo recebe uma lista de diretórios e retorna duas listas: A primeira com os caminhos de todos os arquivos nestes diretórios.
A segunda com um vetor de booleandos indicando se cada arquivo é uma mensagem legítima ou "Spam".


In [ ]:
def processa_diretorios(diretorios):
  rotulos = []
  arquivos = []
  for d in diretorios:
    for a in os.listdir(d):
      arquivos.append(os.path.join(d, a))
      rotulos.append(a.startswith('spm'))
  return arquivos, rotulos

Exemplo:

In [ ]:
arqs, rt = processa_diretorios(['lingspam_public/lemm_stop/part1'])
pd.DataFrame({'arquivo': arqs, 'SPAM': rt}).head()

,arquivo,SPAM
0,lingspam_public/lemm_stop/part1/5-1285msg2.txt,False
1,lingspam_public/lemm_stop/part1/5-1264msg5.txt,False
2,lingspam_public/lemm_stop/part1/3-404msg1.txt,False
3,lingspam_public/lemm_stop/part1/spmsga105.txt,True
4,lingspam_public/lemm_stop/part1/5-1240msg1.txt,False


## Dicionário de palavras

Vamos construir um classificador do tipo "bag of words", que considera apenas a *presença* de uma palavra em um texto, ignorando a sua posição.
Neste, a cada entrada será atribuído um vetor que contém na sua $i$-gésima posição a quantidade de vezes que a palavra de índice $i$ aparece.

Para tanto, em primeiro lugar precisamos construir um dicionário que atribui índices a palavras.
Podemos usar o próprio corpo das mensagens para construir este dicionário.
O código abaixo recebe uma lista de arquivos e um inteiro e retorna dois objetos, um dicionário de palavras->índices e uma lista com palavras.
O inteiro diz a quantidade de palavras a ser extraída (são recuperadas as mais comuns).

In [ ]:
def gera_dicionario(arquivos, corte):
    tudo = []       
    for arquivo in arquivos:    
        with open(arquivo) as a:
            for i, l in enumerate(a):
                if i == 2:  # Mensagem comça a partir da 3a linha
                    palavras = l.split()
                    tudo += palavras
    dicionario = Counter(tudo)
    # Limpa o dicionário: retira tudo que não for texto ou palavras com menos que 2 caracteres
    for palavra in list(dicionario.keys()): 
      if (not palavra.isalpha()) or len(palavra) < 2: 
          del dicionario[palavra]
    # Retém apenas as mais comuns
    dicionario = dicionario.most_common(corte)
    palavra_id = {}
    id_palavra = []
    for i, p in enumerate(dicionario):
      palavra_id[p[0]]=i
      id_palavra.append(p[0])
    return palavra_id, id_palavra

Vamos gerar um dicionário usando os arquivos em ```part1```, ```part2``` e ```part3```.
Serão retidas as 3000 palavras mais comuns.

In [ ]:
arquivosd, rotulos = processa_diretorios(['lingspam_public/lemm_stop/part1', 'lingspam_public/lemm_stop/part2', 'lingspam_public/lemm_stop/part3'])
palavra_id, id_palavra = gera_dicionario(arquivosd, 3000)

Exemplo das palavras extraídas:

In [ ]:
id_palavra[:10]

['language',
 'university',
 'one',
 'linguistic',
 'address',
 'mail',
 'work',
 'order',
 'send',
 'word']

In [ ]:
id_palavra[2]

'one'

## Classificação:

De posse deste dicionário, é possível criar um vetor de características para cada mensagem.

Como descrito acima, este vetor contém na sua $i$-gésima posição a quantidade de vezes que a palavra de índice $i$ aparece.

O código abaixo recebe um caminho para um arquivo e um dicionário de palava->índice e retorna este vetor:

In [ ]:
def gera_vetor(arquivo, palavra_id): 
    vetor = np.zeros(len(palavra_id))
    with open(arquivo) as a:
      for i, l in enumerate(a):
        if i == 2:
          palavras = l.split()
          for palavra in palavras:
            if palavra in palavra_id:
              vetor[palavra_id[palavra]] += 1
    return vetor

Exemplo:

In [ ]:
gera_vetor('lingspam_public/lemm_stop/part4/6-241msg3.txt', palavra_id)

array([2., 1., 1., ..., 0., 0., 0.])

Você deve construir um classificador do tipo Naive Bayes para este conjunto de mensagens.
Use o objeto ```MultinomialNB``` da biblioteca SKLearn.

O método ```fit(X, y)``` recebe uma matriz em ```X``` na qual cada linha é um vetor característico de um objeto e um vetor ```y``` no qual cada coeficiente é a classificação correta do objeto correspondente (ou seja, ```y``` tem tantos coeficientes quanto ```X``` tem de linhas).

Monte a sua matriz ```X``` e o seu vetor ```y``` com os arquivos dos diretórios ```lingspam_public/lemm_stop/part4```, ```lingspam_public/lemm_stop/part5```, ```lingspam_public/lemm_stop/part6``` e ```lingspam_public/lemm_stop/part7```.

In [ ]:
arquivosd, rotulos = processa_diretorios([
                                          'lingspam_public/lemm_stop/part4',
                                          'lingspam_public/lemm_stop/part5',
                                          'lingspam_public/lemm_stop/part6',
                                          'lingspam_public/lemm_stop/part7'])
dados = []
for a in arquivosd:
  dados.append(gera_vetor(a, palavra_id).tolist())

modelo = MultinomialNB()
modelo.fit(dados, rotulos)
  

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

## Avaliação do Classificador.

O método ```predict(X)``` do classificador recebe uma matriz ```X``` na qual cada linha é um vetor característico de um objeto.
Ele retorna um vetor com a classificação prevista para cada objeto.

Mostre a matriz de confusão do classificador que você montou acima quando aplicado às mensagens no diretório ```lingspam_public/lemm_stop/part8```.

In [ ]:
arquivos_teste, rotulos_teste = processa_diretorios(['lingspam_public/lemm_stop/part8'])
dados_teste = []
for a in arquivos_teste:
  dados_teste.append(gera_vetor(a, palavra_id).tolist())

predicoes = modelo.predict(dados_teste)

confusion_matrix(rotulos_teste, predicoes)

array([[233,   8],
       [  0,  48]])